In [6]:
from SRC.foursquare import exploreForesquare, searchForesquare
from SRC.geofunc import *
from SRC.mongofunc import connectMongoColl

from haversine import haversine
# harvesine (lat,long) --> dist in km.

import pandas as pd
from pymongo import MongoClient
import numpy as np

ImportError: cannot import name 'searchForesquare' from 'SRC.foursquare' (/Users/miguelceinos/datamad0120-projects/P3-GeolocationMongoDB/GeolocationMongoDB_Project/SRC/foursquare.py)

In [5]:
test = searchForesquare("party",1,200)

NameError: name 'searchForesquare' is not defined

In [4]:
test["response"][]


{'meta': {'code': 200, 'requestId': '5e4193e42115362ef9f01044'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'SoHo',
  'headerFullLocation': 'SoHo, New York',
  'headerLocationGranularity': 'neighborhood',
  'query': 'coffe',
  'totalResults': 66,
  'suggestedBounds': {'ne': {'lat': 40.742300018000016,
    'lng': -73.97809316185814},
   'sw': {'lat': 40.70629998199998, 'lng': -74.02550683814187}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5703f9a5cd1014db975d2d21',
       'name': 'About Coffee',
       'contact': {},
       'location': {'address': '71 Sullivan St',
        'crossStreet': 'btwn Spring & Broome St',
        'lat': 40.7